# Creating my first graph and running it in a session

The following code creates the graph

In [1]:
import tensorflow as tf

x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

This code does not actually perform any computation. In fact, even the variables are not initialized yet. To evaluate this graph, you need to open a TensorFlow _session_ and use it to initialize the variables and evaluate **f**.

In [2]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print (result)
sess.close()

42


Having to repeat `sess.run()` all the time is a bit cumbersome. Here is the better way.

In [3]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

Instead of manually running the initializer for every single variable, you can use the `global_variables_initializer()` function

In [4]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run() # actually initializer all the variables
    result = f.eval()

Inside Jupyter, you may prefer to create an `InteractiveSession`. It automatically sets itself as the default session, so you don't need a with block ( but need to close the session manually)

In [5]:
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)
sess.close()

42


#### A TensorFlow program is typically split into two parts: 

1. Building a computation graph, called the *construction phase*
2. Running it, *the execution phase*

In [6]:
import tensorflow as tf

# The first part: the construction phase
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x*x*y + y + 2

init = tf.global_variables_initializer()

# The second part: the execution phase
with tf.Session() as sess:
    init.run() 
    result = f.eval()
    print(result)

42


# Managing Graphs

Any node you create is automatically added to the default graph:

In [7]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

You can manage multiple independent graphs by creating a new `Graph` and temporarily making it the default graph inside a `with` block, like so:

In [8]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

In [9]:
x2.graph is graph

True

In [10]:
x2.graph is tf.get_default_graph

False

# Lifecycle of a Node Value

A variable starts its life when its initializer is run, and it ends when the session is closed.

In [11]:
w = tf.constant(3)
x = w + 2
y = x + 5 # y depends on x, which depends on w:  w -> x -> y
z = x * 3 

with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


To evaluate y and z efficiently, w/o evaluating w and x twice as in the previous code, you must ask TensorFlow to evaluate both y and z in just one graph run, as shown in the following code:

In [12]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15
